In [22]:
import torch
from sklearn.datasets import make_classification
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader , TensorDataset

import optuna

In [6]:
X, y = make_classification(n_samples = 1000 ,n_features = 20 , n_classes=2 , random_state= 42)
X_train , X_test , y_train , y_test = train_test_split(X , y , test_size = 0.2 , random_state = 42)

In [9]:
X_train.shape , y_train.shape

((800, 20), (800,))

In [13]:
X_train_tensor = torch.tensor(X_train ,dtype=torch.float32)
y_train_tensor = torch.tensor(y_train ,dtype=torch.float32)
X_test_tensor = torch.tensor(X_test ,dtype=torch.float32)
y_test_tensor = torch.tensor(y_test ,dtype=torch.float32)

In [16]:
X_train_tensor.dtype

torch.float32

In [38]:
class Simple_nn(nn.Module):
    def __init__(self , input_dim , hidden_dim):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim , hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim , 2)
        )

    def forward(self , x):
        return self.network(x)

In [43]:
def objective(trial):

    learning_rate = trial.suggest_loguniform("learning_rate" , 0.0001 , 0.01)
    hidden_dim = trial.suggest_int("hidden_dim" , 16 , 128)

    model = Simple_nn(input_dim = 20 , hidden_dim = hidden_dim)
    createrion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters() , lr = learning_rate)

    train_tensor_dataset = TensorDataset(X_train , y_train)
    test_tensor_dataset = TensorDataset(X_test , y_test)
    
    batch_size = 32
    train_loader = DataLoader(train_tensor_dataset , batch_size = batch_size , shuffle = True)
    test_loader = DataLoader(test_tensor_dataset , batch_size = batch_size , shuffle = True)

    epochs = 20
    for epoch in range(epochs):
        model.train()
        for batch_X , batch_y in train_dataloader:
            outputs = model(batch_X)
            loss = createrion(outputs , batch_y)
    
            loss.backward()
    
            optimizer.step()
            optimizer.zero_grad()
        
    # Validation accuracy
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_X, batch_y in val_loader:
            outputs = model(batch_X)
            _, predicted = torch.max(outputs, 1)
            total += batch_y.size(0)
            correct += (predicted == batch_y).sum().item()

    accuracy = correct/total

    return accuracy

In [44]:
study = optuna.create_study(direction = "maximize")
study.optimize(objective, n_trials=20)

[I 2026-01-12 13:50:43,784] A new study created in memory with name: no-name-715792b2-d3c1-42a0-96a2-d296ce566421
C:\Users\Amal\AppData\Local\Temp\ipykernel_31096\2276595437.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate" , 0.0001 , 0.01)
[W 2026-01-12 13:50:46,238] Trial 0 failed with parameters: {'learning_rate': 0.00048696368640896665, 'hidden_dim': 87} because of the following error: TypeError("'int' object is not callable").
Traceback (most recent call last):
  File "C:\Users\Amal\AppData\Roaming\Python\Python310\site-packages\optuna\study\_optimize.py", line 205, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\Amal\AppData\Local\Temp\ipykernel_31096\2276595437.py", line 10, in objective
    train_tensor_dataset = TensorDataset(X_train , y_train)

TypeError: 'int' object is not callable